# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from google_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
cities_input_file = '../WeatherPy/output_data/cities.csv'
city_df = pd.read_csv(cities_input_file)
display(city_df.head())

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Barrow,71.2906,-156.7887,26.60,74,90,11.50,US,1618627794
1,1,Devils Lake,48.1128,-98.8651,35.01,80,1,6.91,US,1618628019
2,2,Kahului,20.8947,-156.4700,78.80,61,40,9.22,US,1618627767
3,3,Sitka,57.0531,-135.3300,48.20,76,75,1.01,US,1618628113
4,4,Sompeta,18.9333,84.6000,88.29,50,0,5.79,IN,1618627996


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)
heat_map = gmaps.figure()
heat_map

Figure(layout=FigureLayout(height='420px'))

In [9]:
locations = city_df[['Lat','Lng']]
weights = city_df['Humidity']
heat_map.add_layer(gmaps.heatmap_layer(locations,weights=weights))
heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Find ideal weather
# Temp between 80 and 70
cleaned_city_df = city_df[city_df['Max Temp'] < 80]
cleaned_city_df = cleaned_city_df[cleaned_city_df['Max Temp'] > 70]
# Wind Speed less than 10
cleaned_city_df = cleaned_city_df[cleaned_city_df['Wind Speed'] < 10]
# zero cloudiness
cleaned_city_df = cleaned_city_df[cleaned_city_df['Cloudiness'] == 0]
display(cleaned_city_df)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
100,100,Belmonte,-15.8631,-38.8828,73.40,83,0,4.61,BR,1618627931
173,173,Saint-Joseph,-21.3667,55.6167,77.00,83,0,3.44,RE,1618628228
236,236,Eilat,29.5581,34.9482,71.06,34,0,5.06,IL,1618628297
241,241,Umluj,25.0213,37.2685,75.94,70,0,4.85,SA,1618628299
267,267,Trindade,-16.6494,-49.4889,73.40,68,0,4.61,BR,1618628312
330,330,Saint-Philippe,-21.3585,55.7679,77.00,83,0,3.44,RE,1618628370
366,366,Salalah,17.0151,54.0924,77.00,73,0,5.75,OM,1618628366
398,398,Atar,20.5169,-13.0499,71.62,29,0,9.69,MR,1618627775
410,410,Santa Cruz de la Sierra,-17.8000,-63.1667,73.08,77,0,5.08,BO,1618628358
422,422,Dzidzantun,21.2500,-89.0500,77.04,74,0,3.56,MX,1618628461


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Store into hotel_df
hotel_df = cleaned_city_df
lats = hotel_df['Lat'].values
longs = hotel_df['Lng'].values
hotels_json = []
for i in range(len(lats)):
    r = requests.get(f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=name&locationbias=circle:50@{lats[i]},{longs[i]}&key={g_key}&')
    hotels_json.append(r.json())

In [17]:
hotel_names = []
for item in hotels_json:
    hotel_names.append(item['candidates'][0]['name'])
print(hotel_names)

['Aparthotel BahiaTropical', 'Palm Hotel & Spa', 'red Sea hotel', 'HP Red Sea Hotel', 'Hotel Bom Jesus', 'Les Embruns Du Baril', 'Bristol Hotel Salalah', 'Auberge mer et desert', 'TUCABACA HOTEL | BOUTIQUE', 'Hacienda San Francisco Tzacalha', 'Hotel Brisas De Verano', 'Hacienda San Francisco Tzacalha', 'Hotel Valle Verde', 'Palm Hotel & Spa', 'Al Bustan Palace, A Ritz-Carlton Hotel']


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
